# Dec 18-20, 2023: N162 registered coarse parcellations

In [1]:
import os
import numpy as np 
import pandas as pd 
import ants
import json
from tqdm import tqdm 

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
BASE_path = f'/home/govindas/mouse_dataset'

# 172 rois
parcels_172 = (
    f'{BASE_path}/n162_parcellations'
    f'/172_rois_N162_merge_medium_fine_224_0.2_RAS.nii'
)
parcels_172 = ants.image_read(parcels_172)

with open(f'{BASE_path}/n162_parcellations/172_rois.json') as f:
    roi_labels_172 = json.load(f)['labels']['data'][0]['labels']
    roi_labels_172 = pd.DataFrame({
        'roi': [int(k) for k in list(roi_labels_172.keys())],
        'label': list(roi_labels_172.values())
    })

# 56 rois
parcels_56 = (
    f'{BASE_path}/n162_parcellations'
    f'/56_rois_new_N162_coarse_medium_224_noskip_0.2_RAS.nii'
)
parcels_56 = ants.image_read(parcels_56)

roi_labels_56 = pd.read_csv(f'{BASE_path}/n162_parcellations/56_rois.csv', names=['roi', 'label'])

In [3]:
n162_ras_file = f'/home/govindas/mouse_dataset/gabe_symmetric_N162/Symmetric_N162_0.20_RAS.nii.gz'
n162_ras = ants.image_read(n162_ras_file)

In [4]:
def orient_parcels(n162_ras, parcels):
    n = parcels.numpy()
    n = n.transpose(0, 2, 1)
    parcels_xf = n162_ras.new_image_like(n)
    return parcels_xf

In [5]:
parcels_172_ras = orient_parcels(n162_ras, parcels_172)
parcels_172_ras.to_file(f'{BASE_path}/n162_parcellations/172_parcels_RAS.nii.gz')

parcels_56_ras = orient_parcels(n162_ras, parcels_56)
parcels_56_ras.to_file(f'{BASE_path}/n162_parcellations/56_parcels_RAS.nii.gz')

In [6]:
# common brain mask (across subs)
all_files_path = f'{BASE_path}/voxel/all_file_collections'
all_files = os.listdir(all_files_path)

# cmask : common brain mask
for idx, files in tqdm(enumerate(all_files[:])):
    if idx == 0:
        with open(f'{all_files_path}/{files}', 'r') as f:
            ants_cmask = ants.image_read(f.readlines()[1][:-1])
        cmask = ants_cmask.numpy()
    else:
        with open(f'{all_files_path}/{files}', 'r') as f:
            cmask *= ants.image_read(f.readlines()[1][:-1]).numpy()
ants_cmask = ants_cmask.new_image_like(cmask)
ants_cmask.to_filename(
    f'{BASE_path}/voxel/common_brain_mask.nii.gz'
)

116it [00:00, 442.15it/s]


In [7]:
def resample_to_common_mask(cmask_img, parcels_img, name):
    parcels_img_warped = ants.resample_image_to_target(
        image=parcels_img,
        target=cmask_img,
        interp_type='genericLabel',
    )
    parcels_img_warped = parcels_img_warped.new_image_like(
        data=parcels_img_warped.numpy() * cmask_img.numpy()
    )
    fname = f'{BASE_path}/n162_parcellations/{name}_cm.nii.gz'
    print(fname)
    parcels_img_warped.to_filename(
        fname
    )
    print(np.max(np.unique(parcels_img_warped.numpy())))
    return fname

fname_172 = resample_to_common_mask(ants_cmask, parcels_172_ras, '172_parcels_RAS')
fname_56 = resample_to_common_mask(ants_cmask, parcels_56_ras, '56_parcels_RAS')

/home/govindas/mouse_dataset/n162_parcellations/172_parcels_RAS_cm.nii.gz
172.0
/home/govindas/mouse_dataset/n162_parcellations/56_parcels_RAS_cm.nii.gz
56.0


In [8]:
def roi_labels(mask_file, name):
    cmd = (
        f'3dROIstats -overwrite '
        f'-quiet '
        f'-mask {mask_file} '
        f'{mask_file} > {BASE_path}/n162_parcellations/{name}_roi_labels.txt'
    )
    os.system(cmd)
    return None

roi_labels(fname_172, '172')
roi_labels(fname_56, '56')